Install Requirements ()

In [1]:
!pip install autogen-agentchat autogen-ext[openai] -q
!pip install groq -q
!pip install arxiv -q
!pip install PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-sdk 1.16.0 requires opentelemetry-api==1.16.0, but you have opentelemetry-api 1.30.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import arxiv
from autogen_core.tools import FunctionTool
from autogen_agentchat.conditions import MaxMessageTermination

In [3]:
import os
from getpass import getpass

tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

Enter GROQ_API_KEY here: ··········


In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)
assistant = AssistantAgent("assistant", model_client=custom_model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("DONE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?")
await Console(stream)

---------- user ----------
the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?
---------- assistant ----------
I'm a large language model, I don't have the capability to directly receive or store files, including PDFs. I can process text-based input, but I don't have a file upload or download mechanism.

If you'd like to share the content of the PDF with me, you can try copying and pasting the text into this chat window. If the text is too long, you can break it up into smaller chunks and send them to me one at a time. I'll do my best to assist you with the content.


In [18]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "Multi-agent LLM systems",
  max_results = 2,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in client.results(search):
  # print(f"Entry ID: {result.entry_id}")
  print(f"\nTitle: {result.title}")
  # print(f"\nAuthors: {', '.join([a.name for a in result.authors])}")
  # print(f"\nPublished: {result.published}")
  # print(f"\nUpdated: {result.updated}")
  print(f"\nSummary: {result.summary}")
  # print(f"\nComment: {result.comment}")
  # print(f"\nJournal Reference: {result.journal_ref}")
  # print(f"\nDOI: {result.doi}")
  # print(f"\nPrimary Category: {result.primary_category}")
  # print(f"\nCategories: {result.categories}")
  # print(f"\nPDF URL: {result.pdf_url}\n")


Title: Emergence of Fermi's Golden Rule in the Probing of a Quantum Many-Body System

Summary: Fermi's Golden Rule (FGR) is one of the most impactful formulas in quantum
mechanics, providing a link between easy-to-measure observables - such as
transition rates - and fundamental microscopic properties - such as density of
states or spectral functions. Its validity relies on three key assumptions: the
existence of a continuum, an appropriate time window, and a weak coupling.
Understanding the regime of validity of FGR is critical for the proper
interpretation of most spectroscopic experiments. While the assumptions
underlying FGR are straightforward to analyze in simple models, their
applicability is significantly more complex in quantum many-body systems. Here,
we observe the emergence and breakdown of FGR, using a strongly interacting
homogeneous spin-$1/2$ Fermi gas coupled to a radio-frequency (rf) field.
Measuring the transition probability into an outcoupled internal state, we map

In [ ]:
paper = next(arxiv.Client().results(arxiv.Search(id_list=["1605.08386v1"])))
print(paper.get_short_id())

In [ ]:
import PyPDF2
import openai

# Step 1: Extract text from the PDF
pdf_path = "/content/downloaded-paper.pdf"  # Adjust the path as needed
paper_text = ""
with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            paper_text += page_text + "\n"

# Step 2: Split the text into manageable chunks
def chunk_text(text, max_length=2000):
    """Splits text into chunks of approximately max_length characters."""
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]


chunks = chunk_text(paper_text, max_length=2000)


In [ ]:
print(chunks[1])

**1. Research Agent (Paper Retriever)**

    Purpose: Searches for and downloads relevant academic papers from databases (e.g., Google Scholar, ArXiv, Semantic Scholar).
    Responsibilities:
    1.   Querying research databases with relevant keywords.
    2.   Downloading full-text articles or abstracts.
    3.   Storing references in a structured manner (e.g., BibTeX or JSON format).

In [75]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)
    print("************ calling")
    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
            }
        )

    return results

In [76]:

arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

In [77]:
arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)

**2. Reader Agent (Document Processor)**

    Purpose: Reads, parses, and extracts key information from research papers.
    Responsibilities:
    1.   Extracting abstracts, keywords, and main sections (introduction, methodology, results, discussion).
    2.   Identifying key arguments, methodologies, and findings.
    3.   Converting scanned PDFs to machine-readable text (OCR, if needed).

In [78]:
report_agent = AssistantAgent(
    name="Report_Agent",
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to get the papers info mostly only titles from another agent you're gonna select only recent one and give it back to the user, user might ask questions you still need to answer their questions.",
)

**5. Human Feedback Agent (Interactive Editor)**

    Purpose: Allows human users to provide feedback and modify the generated text.
    Responsibilities:
        1.   Displaying the draft for user review.
        2.   Accepting edits, comments, and suggestions.
        3.   Incorporating changes into the final version.

In [79]:
user_proxy = UserProxyAgent("user_proxy", input_func=input)

Finally, we will create a team of agents and configure them to perform the tasks.

In [10]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    participants=[arxiv_search_agent, report_agent, user_proxy], termination_condition=termination
)

NameError: name 'arxiv_search_agent' is not defined

In [82]:
await Console(
    team.run_stream(
        task="Give me name of some research papers reagrding no code tools",
    )
)

---------- user ----------
Give me name of some research papers reagrding no code tools
************ calling---------- Arxiv_Search_Agent ----------

[FunctionCall(id='call_asaz', arguments='{"query": "no code tools research papers", "max_results": 5}', name='arxiv_search')]
---------- Arxiv_Search_Agent ----------
[FunctionExecutionResult(content='[{\'title\': "Ease on Down the Code: Complex Collaborative Qualitative Coding Simplified with \'Code Wizard\'"}, {\'title\': \'Bridging the Gap: A Survey and Classification of Research-Informed Ethical Hacking Tools\'}, {\'title\': \'A Tool for In-depth Analysis of Code Execution Reasoning of Large Language Models\'}, {\'title\': \'Deep Learning-Based Video Coding: A Review and A Case Study\'}, {\'title\': \'Code Swarm: A Code Generation Tool Based on the Automatic Derivation of Transformation Rule Set\'}]', call_id='call_asaz', is_error=False)]
---------- Arxiv_Search_Agent ----------
[{'title': "Ease on Down the Code: Complex Collaborative

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give me name of some research papers reagrding no code tools', type='TextMessage'), ToolCallRequestEvent(source='Arxiv_Search_Agent', models_usage=RequestUsage(prompt_tokens=1689, completion_tokens=26), content=[FunctionCall(id='call_asaz', arguments='{"query": "no code tools research papers", "max_results": 5}', name='arxiv_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Arxiv_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': "Ease on Down the Code: Complex Collaborative Qualitative Coding Simplified with \'Code Wizard\'"}, {\'title\': \'Bridging the Gap: A Survey and Classification of Research-Informed Ethical Hacking Tools\'}, {\'title\': \'A Tool for In-depth Analysis of Code Execution Reasoning of Large Language Models\'}, {\'title\': \'Deep Learning-Based Video Coding: A Review and A Case Study\'}, {\'title\': \'Code Swarm: A Code Generation T

In [28]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "assistant",
    model_client=custom_model_client,
    system_message="help the user with his questions",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
# handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=7)
combined_termination = max_messages_termination | text_termination

# Create a single-agent team.
user_proxy = UserProxyAgent("user_proxy", input_func=input)
lazy_agent_team = RoundRobinGroupChat([lazy_agent, user_proxy], termination_condition=combined_termination)

# Run the team and stream to the console.
task = "What is the weather in New York?"
await Console(lazy_agent_team.run_stream(task=task))

---------- user ----------
What is the weather in New York?
---------- assistant ----------
I'm a large language model, I don't have have access to real-time information, so I can't provide you with the current weather in New York. However, I can suggest some ways for you to find out the current weather in New York:

1. **Check online weather websites**: You can visit websites like weather.com, accuweather.com, or wunderground.com to get the current weather conditions in New York.
2. **Use a weather app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get real-time weather updates for New York.
3. **Check social media**: You can check the official social media accounts of the National Weather Service (NWS) or the New York City government to get updates on the weather.
4. **Turn on the TV**: You can watch the local news or weather channel on TV to get the current weather conditions in New York.

If you're looking for general information a

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=49, completion_tokens=280), content="I'm a large language model, I don't have have access to real-time information, so I can't provide you with the current weather in New York. However, I can suggest some ways for you to find out the current weather in New York:\n\n1. **Check online weather websites**: You can visit websites like weather.com, accuweather.com, or wunderground.com to get the current weather conditions in New York.\n2. **Use a weather app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get real-time weather updates for New York.\n3. **Check social media**: You can check the official social media accounts of the National Weather Service (NWS) or the New York City government to get updates on the weather.\n4. **Turn on the T

In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

# Note: This example uses mock tools instead of real APIs for demonstration purposes
# def search_web_tool(query: str) -> str:
#     if "2006-2007" in query:
#         return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
#         Udonis Haslem: 844 points
#         Dwayne Wade: 1397 points
#         James Posey: 550 points
#         ...
#         """
#     elif "2007-2008" in query:
#         return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
#     elif "2008-2009" in query:
#         return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
#     return "No data found."


# def percentage_change_tool(start: float, end: float) -> float:
#     return ((end - start) / start) * 100




model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        ResearchAgent: Paper Retriever
        ReaderAgent: Document Processor
        SummarizationAgent: Content Extractor

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings.
    """,
)

# web_search_agent = AssistantAgent(
#     "WebSearchAgent",
#     description="An agent for searching information on the web.",
#     tools=[search_web_tool],
#     model_client=model_client,
#     system_message="""
#     You are a web search agent.
#     Your only tool is search_tool - use it to find information.
#     You make only one search call at a time.
#     Once you have the results, you never do calculations based on them.
#     """,
# )

# data_analyst_agent = AssistantAgent(
#     "DataAnalystAgent",
#     description="An agent for performing calculations.",
#     model_client=model_client,
#     tools=[percentage_change_tool],
#     system_message="""
#     You are a data analyst.
#     Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
#     If you have not seen the data, ask for it.
#     """,
# )




# 1

def research_arxiv_tool(query: str) -> str:
    import arxiv

    # Construct the default API client.
    client = arxiv.Client()

    # Search for the 2 most recent articles matching the query.
    search = arxiv.Search(
        query=query,
        max_results=2,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    results_list = []

    for result in client.results(search):
        results_list.append(f"Title: {result.title}\nSummary: {result.summary}\n")

    return "\n".join(results_list) if results_list else "No results found."

research_agent = AssistantAgent(
    "ResearchAgent",
    description="You are an AI-powered assistant that extracts concise and effective search queries for academic research. Given a user input, it refines the request into a well-structured query that optimizes the ArXiv search tool for relevant results.",
    tools=[research_arxiv_tool],
    model_client=model_client,
    system_message="""
    You are a research query optimization agent.
    Your only task is to extract a well-formed and precise search query from user input.
    You do not summarize, analyze, or modify the research papers.
    Your job is only to construct the best possible query for the research_arxiv_tool.
    Ensure the query is:
    - Clear and relevant to the user’s request.
    - Free of unnecessary words or ambiguity.
    - Structured to yield useful academic results.

    If the user's input is too vague, return a refined general query that would still provide meaningful results.
    Never generate responses unrelated to query refinement.
    """,
)

reader_agent = AssistantAgent(
    "ReaderAgent",
    description="An AI-powered research assistant designed to analyze academic papers and extract key information. It helps users quickly understand the core content of a research paper without reading the full document.",
    model_client=model_client,
    system_message="""
    You are an intelligent document processing agent specialized in academic research.
    Your task is to **read, parse, and extract key information** from research papers.
    You DO NOT summarize arbitrarily—your goal is structured extraction of information.

    For every research paper you process, extract and return:
    - **Title**
    - **Abstract**
    - **Keywords**
    - **Main Sections**: (Introduction, Methodology, Results, Discussion)
    - **Key Arguments & Findings**
    - **Notable Limitations (if mentioned in the paper)**

    Maintain accuracy and structure. Do not add opinions or assumptions.
    If a section is missing or unclear, mention it without making up content.
    Your job is purely **document analysis and structured extraction**, not interpretation.
    """,
)

summarization_agent = AssistantAgent(
    "SummarizationAgent",
    description="An AI-powered research assistant that generates structured summaries of research papers. It extracts key insights, highlights major contributions, and identifies gaps in the literature while organizing findings into thematic categories.",
    model_client=model_client,
    system_message="""
    You are an advanced research summarization agent.
    Your job is to generate **structured summaries** of research papers and extract key insights.
    You DO NOT simply copy abstracts—you summarize in an organized and meaningful way.

    For every paper, produce a structured summary including:
    - **Title**
    - **Core Summary**: (A concise, 3-5 sentence explanation of the paper)
    - **Major Contributions**: (What new insights, methods, or findings does this paper offer?)
    - **Identified Gaps**: (What areas remain unexplored or need further research?)
    - **Categorized Findings**: (If possible, organize the insights into themes or subtopics)

    Your goal is to make research **accessible and structured** while preserving accuracy.
    Avoid unnecessary details or overly technical jargon unless necessary.
    """,
)


selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


team = SelectorGroupChat(
    [planning_agent, research_agent, reader_agent, summarization_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

task = "I am researching the latest advancements in Multi-Agent Large Language Models (LLMs). Specifically, I want to understand how multiple LLMs can collaborate to solve complex tasks, including coordination strategies, communication methods, and distributed problem-solving. Please find relevant research papers, extract key insights, and summarize major contributions and gaps in this field."

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
I am researching the latest advancements in Multi-Agent Large Language Models (LLMs). Specifically, I want to understand how multiple LLMs can collaborate to solve complex tasks, including coordination strategies, communication methods, and distributed problem-solving. Please find relevant research papers, extract key insights, and summarize major contributions and gaps in this field.
---------- PlanningAgent ----------
To accomplish this task, I will break it down into smaller subtasks and delegate them to my team members. Here's the plan:

1. **ResearchAgent** : Retrieve research papers on Multi-Agent Large Language Models (LLMs), focusing on collaboration, coordination strategies, communication methods, and distributed problem-solving.
2. **ReaderAgent** : Process the retrieved papers, identifying and annotating relevant sections that discuss the latest advancements, challenges, and gaps in multi-agent LLMs.
3. **SummarizationAgent** : Extract key insights

In [27]:
await team.reset()